In [12]:
import time
from matplotlib import pyplot as plt
import numpy as np
import dlib
import cv2

from keras.models import load_model
from keras.models import model_from_json

%matplotlib inline
from IPython.display import clear_output
import tensorflow as tf


import util

import time
import posenet

ModuleNotFoundError: No module named 'dlib'

# Pose Estimation network

In [7]:
#Pose estimation

modelP=50
siz=300

scale_factor=1

#Load the model
with tf.Session() as sess:
    model_cfg, model_outputs = posenet.load_model(modelP, sess)
    output_stride = model_cfg['output_stride']

def LeftCheck(cap):
    e,t = cap.read()
    #Resizing to reduce computation requirement
    t = cv2.resize(t, (300,300), interpolation=cv2.INTER_AREA)
    
    #declaring input image, output image, and output scale 
    input_image, display_image, output_scale = posenet.str_im(
        t, scale_factor=scale_factor, output_stride=output_stride)

    #Sending the input frame for pose estimation
    heatmaps_result, offsets_result, displacement_fwd_result, 
        displacement_bwd_result = sess.run(
        model_outputs,
        feed_dict={'image:0': input_image}
    )

    #Processing the output of the pose estimation network
    pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
        heatmaps_result.squeeze(axis=0),
        offsets_result.squeeze(axis=0),
        displacement_fwd_result.squeeze(axis=0),
        displacement_bwd_result.squeeze(axis=0),
        output_stride=output_stride,
        max_pose_detections=1,
        min_pose_score=0.15)

    keypoint_coords *= output_scale

    nose=keypoint_coords[0,0][0]
    left=keypoint_coords[0,9][0]
    right=keypoint_coords[0,10][0]

    if(keypoint_scores[0,0]>0.1 and keypoint_scores[0,9]>0.1 and  left<nose):
        return 'Left' 
        
    elif(keypoint_scores[0,0]>0.1 and keypoint_scores[0,9]>0.1):
        
        return ([int(keypoint_coords[0,10][0]), int(keypoint_coords[0,10][1])])
    #else:
       # return('not Av')
       # return 'Right'
    
def poser(cap):
    e,t = cap.read()
    t = cv2.resize(t, (300,300), interpolation=cv2.INTER_AREA)
    
    input_image, display_image, output_scale = posenet.str_im(
        t, scale_factor=scale_factor, output_stride=output_stride)

    heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
        model_outputs,
        feed_dict={'image:0': input_image}
    )


    pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
        heatmaps_result.squeeze(axis=0),
        offsets_result.squeeze(axis=0),
        displacement_fwd_result.squeeze(axis=0),
        displacement_bwd_result.squeeze(axis=0),
        output_stride=output_stride,
        max_pose_detections=1,
        min_pose_score=0.15)

    keypoint_coords *= output_scale

    nose=keypoint_coords[0,0][0]
    left=keypoint_coords[0,9][0]
    right=keypoint_coords[0,10][0]

    if(keypoint_scores[0,0]>0.1 and keypoint_scores[0,10]>0.1 and  right<nose and left<nose):
        return 'BothUp'
    
    elif(keypoint_scores[0,0]>0.1 and keypoint_scores[0,9]>0.1 and  left<nose):
        return 'L' 
        
    elif(keypoint_scores[0,0]>0.1 and keypoint_scores[0,10]>0.1 and  right<nose):
        return 'R'   
    else:
        return 'B'


AttributeError: module 'tensorflow' has no attribute 'Session'

# Converts to binary image

In [4]:

def sketch_transform(image):
    try:
        image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(image_grayscale, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    except:
        mask=image
    return mask

In [ ]:
# Character recognition

In [5]:


import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.models import model_from_json

topcornY=0
topcornX=0

#Loading model and loading the pretrained weights into it
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

y_value = []
  
loaded_model.load_weights('model.h5')

model = loaded_model


print('Model successfully loaded')


cropped_dir_path = "tmpCropped/"
xlabels=[]
ylabels=[]
flag=0

characters = [0,1,2,3,4,5,6,7,8,9,'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']


Model successfully loaded


# Classifies the cell data

In [6]:


def classify(new_img):
        image = new_img
        height, width, depth = image.shape
        #print(height, width,depth)

        #resizing the image to find spaces better
        image = cv2.resize(image, dsize=(width*5,height*4), interpolation=cv2.INTER_CUBIC)
        #grayscale
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)


        #binary
        ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)


        #dilation
        kernel = np.ones((5,5), np.uint8)
        img_dilation = cv2.dilate(thresh, kernel, iterations=1)


        #adding GaussianBlur
        gsblur=cv2.GaussianBlur(img_dilation,(5,5),0)


        #find contours (which are the individual characters in a cell)
        im2,ctrs, hier = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        m = list()
        #sort contours
        sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
        pchl = list()
        for i, ctr in enumerate(sorted_ctrs):
            # Get bounding box around the character
            x, y, w, h = cv2.boundingRect(ctr)
            # Getting ROI
            roi = image[y:y+h, x:x+w]
            try:
                
                #Adding a black border around the character to make sure it's not covering the entire image
                #This is because the classifier CNN was trained on images which had sufficient black space
                #Around them
                roi = cv2.copyMakeBorder(
                                     roi, 
                                     15, 
                                     15, 
                                     45, 
                                     45, 
                                     cv2.BORDER_CONSTANT, 
                                     value=[255, 255, 255]
                                  )
                
                #Resizing in accordance to CNN's input dimensions
                roi = cv2.resize(roi, dsize=(28,28), interpolation=cv2.INTER_CUBIC)
            except:
                continue
            
            #supporting the input dimension for CNN
            roi = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
            
            #Normalizing the pixel values
            roi = np.array(roi)

            t = np.copy(roi)
            t = t / 255.0
            t = 1-t
            t = t.reshape(1,784)
            
            #CNN prediction
            pred = model.predict_classes(t)
            
            #collecting all the characters inside the cell
            pchl.append(pred)



        pcw = list()
        #interp = 'bilinear'
        for i in range(len(pchl)):

            pcw.append(str(characters[pchl[i][0]]))
                        
            
        
        predstring = ''.join((pcw))
        y_value.append(predstring)

        print('Predicted Value: '+predstring)
        
        
def detect(img):
    flag = 0
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    idx = 0
    (thresh, img_bin) = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # Defining a kernel length (floored)
    kernel_length = np.array(img).shape[1]//80

    # A verticle kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
    verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
    
    # A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
    hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
    # A kernel of (3 X 3) ones.
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    # Morphological operation to detect vertical lines from an image
    img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
    verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
    cv2.imwrite('Ver.png',verticle_lines_img)
    #plt.imshow(verticle_lines_img, cmap = "gray")
    # Morphological operation to detect horizontal lines from an image
    img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
    horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
    cv2.imwrite('Hor.png',horizontal_lines_img)
    # Weighting parameters, this will decide the quantity of an image to be added to make a new image.
    alpha = 0.5
    beta = 1.0 - alpha
    # This function helps to add two image with specific weight parameter to get a third image as summation of two image.
    img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
    
    #finds all corners
    img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
    (thresh, img_final_bin) = cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    #saves all the horizontal and vertical components
    cv2.imwrite('Cells.png', img_final_bin)
    
    
    def sort_contours(cnts, method="left-to-right"):
        # initialize the reverse flag and sort index
        reverse = False
        i = 0

        # handle if we need to sort in reverse
        if method == "right-to-left" or method == "bottom-to-top":
            reverse = True

        # handle if we are sorting against the y-coordinate rather than
        # the x-coordinate of the bounding box
        if method == "top-to-bottom" or method == "bottom-to-top":
            i = 1

        # construct the list of bounding boxes and sort them from top to
        # bottom
        boundingBoxes = [cv2.boundingRect(c) for c in cnts]
        
        #iterates through the contours and sorts them in left to right order
        (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
            key=lambda b:b[1][i], reverse=reverse))

        # return the list of sorted contours and bounding boxes
        return (cnts, boundingBoxes)

    # Find contours for image, which will detect all the boxes
    im2, contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Sort all the contours by top to bottom.
    (contours, boundingBoxes) = sort_contours(contours, method="left-to-right")
    
    #The first contour is the entire box, so we skip that
    contours = iter(contours)
    next(contours)
    
    for c in contours:
            # Returns the location and width,height for every contour
            if(flag==0):
                flag=1
                
                #print(cv2.boundingRect(c))
                continue

            x, y, w, h = cv2.boundingRect(c)

    # If the box height is greater then 10, widht is >10, then only save it as a box 
            if (w > 50 and h > 50):# and w > 1.05*h:
                idx += 1
                topcornX, topcornY = x+w,y
                new_img = img[y+5:y+h-5, x+4:x+w-4]
                new_img = cv2.bitwise_not(new_img)

                #Find out the contents of the image
                classify(new_img) 
                
                #save individual cells
                cv2.imwrite(cropped_dir_path+str(idx) + '.png', new_img)
                            #enter input image here

    return idx
    #plt.imshow(img)


# Affine transformation to undistort tabular data

In [7]:


def GeoTX(im):

    im = cv2.imread("inputCam.png")
    numcards=1

    #grey
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    
    #binary
    flag, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #plt.imshow(thresh)
    cv2.imwrite('distorted.png', thresh)
    
    
    image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #only taking the parent contours (The outer ones, which will include the corners)
    contours = sorted(contours, key=cv2.contourArea,reverse=True)[:numcards]  

    for i in range(numcards):
        card = contours[i]
        
        #measures perimeter of the contour
        peri = cv2.arcLength(card,True)
        
        #approximates the contour's shape with less number of corners within 2%. 
        #because the contour can have more than 4 corners due to human errors
        approx = cv2.approxPolyDP(card,0.02*peri,True)
        
 
    #converting to numpy array to calculate means and other stats. 
    #squeeze reduces the extra outer dimension
    approx = np.squeeze(approx.astype(float))
    approx = np.float32(approx)
    
    #finds means of the x,y points of a corner
    means = np.mean(approx,1)
    inds = means.argsort()
    #arranges the corner points in a cyclic order (for example : TopL, TopR, BotR, BotL)
    approx = approx[inds]
    
    #finds the minimum bounding rectangle for the big box
    img = thresh.copy()
    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

    #holds the corners of the minimum bounding rectangle
    dst = np.float32([ [x,y],[x,y+h], [x+w,y],[x+w,y+h]])
    
    #holds the transformation from the original corners to the future corners
    transform = cv2.getPerspectiveTransform(approx, dst)

    #warps the image using the transformation
    warp = cv2.warpPerspective(im,transform,((thresh.transpose().shape)))
    
    #resizing the wrapped image
    warp = cv2.resize(warp, (250,250), interpolation=cv2.INTER_AREA)
    
    cv2.imwrite('crop.png',warp)
    a = detect(warp)

    return a

# Designs the graphs

In [8]:


def graphing(im):
    
   
    sizeOfCell = GeoTX(im)
    
    #Correcting the order of the data read
    y_value.reverse()
    
    values=[]
    labels=[]
    colours=[]
    
    #remove this if error
    y_pos=[]
    
    #The first half of the read data are the numbers but they are in string format
    values = (y_value[0:int(sizeOfCell/2)])


    '''THESE VALUES ARE ONLY VALID FOR BAR GRAPH BECAUSE RIGHT SIDE IS CONVERTED TO INT'''
    try:
        values = [ int(x) for x in values ]

    except:
        print('Character detected. Please use integer')
    
    #The second half of the read data are the labels
    labels = (y_value[int(sizeOfCell/2):sizeOfCell])
       
        # -*- coding: utf-8 -*-


    #Just some good ol matplotlib bar graphs
    y_pos = np.arange(len(labels))
    
    colours = np.random.rand(int(sizeOfCell/2),3)

    plt.bar(y_pos, values, align='center', color= colours)
    plt.xticks(y_pos, labels,fontsize=30)
    plt.yticks(fontsize=20)

    # naming the x-axis 
    plt.xlabel('x - axis') 
    # naming the y-axis 
    plt.ylabel('y - axis') 
    # plot title 
    plt.title('Bar Chart') 

    # function to show the plot
    #bbox_inches='tight' 
    plt.savefig('bar2.jpg')#, bbox_inches='tight', transparent=True)
    #plt.show()
    #LoadedGraph = cv2.imread('CosAndSin.jpg')
    LoadedGraph = cv2.imread('bar2.jpg')
    #LoadedGraph = cv2.cvtColor(LoadedGraph, cv2.COLOR_BGR2GRAY)
    return LoadedGraph
    

# Correlation tracker and the main control block

In [12]:


y_value = []
# this variable will hold the coordinates of the mouse click events.
mousePoints = []

def mouseEventHandler(event, x, y, flags, param):
    # references to the global mousePoints variable
    global mousePoints

    # if the left mouse button was clicked, record the starting coordinates.
    if event == cv2.EVENT_LBUTTONDOWN:
        mousePoints = [(x, y)]

    # when the left mouse button is released, record the ending coordinates.
    elif event == cv2.EVENT_LBUTTONUP:
        mousePoints.append((x, y))

# create the video capture.
video_capture = cv2.VideoCapture(0)

# create a named window in OpenCV and attach the mouse event handler to it.
cv2.namedWindow("Webcam stream")
cv2.setMouseCallback("Webcam stream", mouseEventHandler)

# initialize the correlation tracker.
tracker = dlib.correlation_tracker()

# this is the variable indicating whether to track the object or not.
tracked = False

#coordinates initialized. 'z' is for size/scale of overlayed visualization
x,y,x1,y1,overl,z = 0,0,0,0,0,0
zig=[0,0]
frY,frX = [0,0]

graph = cv2.imread('bar.jpg')
stick,flagg=0,0
while True:
    # start capturing the video stream.
    ret, frame = video_capture.read()

    if ret:
        image = frame

        # if we have two sets of coordinates from the mouse event, draw a rectangle.
        if len(mousePoints) == 2:
            cv2.rectangle(image, mousePoints[0], mousePoints[1], (0, 255, 0), 2)
            dlib_rect = dlib.rectangle(mousePoints[0][0], mousePoints[0][1], mousePoints[1][0], mousePoints[1][1])

        # tracking in progress, update the correlation tracker and get the object corner position.
        if tracked == True and overl==0:
            tracker.update(image)
            track_rect = tracker.get_position()
            x  = int(track_rect.left())
            y  = int(track_rect.top())
            x1 = int(track_rect.right())
            y1 = int(track_rect.bottom())
            cv2.rectangle(image, (x-3, y-3), (x1+3, y1+3), (0, 0, 255), 2)
            
            #This function gets a binary image of the rectangle ROI
            transform = sketch_transform(image[y :y1, x : x1])
            
            #Conversion to 3 channels to put back on original image (streaming) [Grayscale can't be overlayed on RGB]
            #Try-Except block in case the ROI goes out of frame
            try:
                sketcher_rect_rgb = cv2.cvtColor(transform, cv2.COLOR_GRAY2RGB)
                image[ y :y1, x : x1] = sketcher_rect_rgb
            except:
                image=image

        #if the visualization has been overlayed.
        if overl == 1 :
            
            #This function checks if left hand is above the nose in order to stick
            #returns the coordinates of right hand as long as left is below the nose
            #Short for "right"
            rig = LeftCheck(video_capture)
            try:
                g1 = cv2.resize(graph, (z,z), interpolation = cv2.INTER_AREA)
                flagg=0
            except:
                flagg=1
                print('graph not loaded')
                clear_output()               
            
            if(stick==0 and flagg==0):
                
                #Checking if left hand is above the nose -> User wants to stick the graph  
                
                if(rig=='Left'): #if the user wants to stick the graph
                    try:
                        stick=1
                        frY,frX = zig[0],zig[1]
                        image[frY:frY+z, frX:frX+z] = g1
                        print('STICKEY!')
                        #Giving user some time to droop their left hand down
                        time.sleep(1)
                        
                    except:
                        image=image
                
                #If user is still moving the graph around with their right
                else:
                    try:
                        zig[0],zig[1] = rig[0],rig[1]
                        image[rig[0]:rig[0]+z, rig[1]:rig[1]+z] = g1
                        
                #incase the right hand goes out of frame, stick it back at top left corner     
                    except:
                        image[0:0+z, 0:0+z] = g1


            #once graph is stick-ed, then start interaction (zoom in/out)
            if (stick==1):
               
                #This tells if right or left is above the nose to zoom in/out
                PosVal = poser(video_capture)
                
                if(PosVal=='B'):
                    z = z
                elif(PosVal=='R'):
                    #changing scales
                    z = z+20
                elif(PosVal== 'L'):
                    if(z>10):
                        z = z-20                    
                try:
                    g1 = cv2.resize(graph, (z,z), interpolation = cv2.INTER_AREA)
                    image[frY:frY+z, frX:frX+z] = g1
                except:
                    image=image
            
            
        cv2.imshow("Webcam stream", image)

    # capture the keyboard event in the OpenCV window.
    ch = 0xFF & cv2.waitKey(1)

    # press "r" to stop tracking and reset the tracking variables.
    if ch == ord("r"):
        clear_output()
        mousePoints = []
        y_value = []
        overl=0
        tracked = False
        #freeze the last scale value for the overlay
        z = int((x1-x+y1-y)/2)
        
    # press "t" to start tracking the currently selected object/area.
    if ch == ord("t"):
        #Making sure a proper ROI has been selected
        if len(mousePoints) == 2:
            tracker.start_track(image, dlib_rect)
            tracked = True
            #can get rid of the manual rectangle now
            mousePoints = []
            
    #Press "d" to detect/process ROI
    if ch == ord("d"):
        clear_output()
        imagePreFlip = image[y : y1, x : x1]
        imagePostFlip = cv2.bitwise_not(imagePreFlip)

        try:
            graph = graphing(imagePostFlip)
            overl = 1
        except:
            print('error in Graphing')
      

    # press "q" to quit the program.
    if ch == ord('q'):
        break

# cleanup.


video_capture.release()
cv2.destroyAllWindows()